<a href="https://colab.research.google.com/github/AgustinaInsua/TP-LABO-GRUPO4/blob/main/pruebakmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importar las librerias

In [6]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from scipy.spatial import ConvexHull
from scipy.spatial import QhullError



ModuleNotFoundError: No module named 'pandas'

Leer el archivo .csv

In [5]:
url = 'https://github.com/AgustinaInsua/TP-LABO-GRUPO4/raw/main/info2018.csv'
nrows_to_read = 10000  # Cambia este valor al número de filas que desees cargar
data = pd.read_csv(url, nrows=nrows_to_read)

data


NameError: name 'pd' is not defined

In [ ]:
# Función para asignar una región a cada provincia
def assign_region(province):
    if province in ['Corrientes', 'Chaco', 'Misiones', 'Formosa']:
        return 'Nordeste-Litoral'
    elif province in ['Tucumán', 'Jujuy', 'Salta', 'Catamarca', 'Santiago del Estero']:
        return 'Noroeste'
    elif province in ['San Luis', 'San Juan', 'Mendoza', 'La Rioja']:
        return 'Cuyo'
    elif province in ['Neuquén', 'Río Negro', 'Tierra del Fuego', 'Santa Cruz', 'Chubut']:
        return 'Patagonia'
    elif province in ['Buenos Aires', 'CABA']:
        return 'Buenos Aires'
    else:
        return 'Pampeana'
# Limpiar las columnas de cadena para eliminar las comillas dobles
string_columns = ['provincia_nombre']  # Agrega aquí las columnas que son cadenas
for column in string_columns:
    data[column] = data[column].str.replace('"', '')  # Eliminar comillas dobles

# Aplicamos la función a la columna 'PROVINCE' y guardamos el resultado en una nueva columna 'REGION'
data['region'] = data['provincia_nombre'].apply(assign_region)
data

In [ ]:
ocurrences = data['region'].value_counts()
print(ocurrences)

In [ ]:
columns_to_drop = ['provincia_nombre', 'departamento_nombre', 'grupo_edad_desc']
data = data.drop(columns=columns_to_drop)
data

In [ ]:
# Asignaciones para columnas no númericas
assignment_mapping = {
    'evento_nombre': {
        'Bronquiolitis en menores de 2 años': 2,
        'Enfermedad tipo influenza (ETI)': 3,
        'Neumonía': 1
    },
    'region': {
        'Buenos Aires': 1,
        'Pampeana': 2,
        'Noroeste': 3,
        'Cuyo': 4,
        'Patagonia': 5,
        'Nordeste-Litoral': 6
    }
}

# Limpia valores de las columnas antes de asignar
string_columns = ['evento_nombre']
for column in string_columns:
    data[column] = data[column].str.strip('"')  # Elimina comillas
    data[column] = data[column].str.strip()     # Elimina espacios en blanco

data.fillna(0, inplace=True)

# Aplica las asignaciones
for column, mapping in assignment_mapping.items():
    data[column] = data[column].map(mapping)
data



In [7]:
'''
n_clusters: En el paso de aplicar K-Means calcularemos
la cantidad de clusters con el Método del codo
'''
wcss=[]
for i in range(2,10):
  kmeans = KMeans(i, n_init=100)
  kmeans.fit(data)
  wcss_iter = kmeans.inertia_
  wcss.append(wcss_iter)

number_clusters = range(2,10)
plt.plot(number_clusters, wcss, marker='*', markersize=10, color='green')
plt.title('Número optimo de clusters')
plt.xlabel('Número de clusters')
plt.ylabel('WCSS')

NameError: name 'KMeans' is not defined

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score

silhouette_scores = []

for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, n_init=100, random_state=0)
    cluster_labels = kmeans.fit_predict(data)

    silhouette_avg = silhouette_score(data, cluster_labels)
    silhouette_scores.append(silhouette_avg)

    # Calcular las puntuaciones de silueta para cada muestra
    sample_silhouette_values = silhouette_samples(data, cluster_labels)

plt.figure(figsize=(8, 6))
plt.plot(range(2, 10), silhouette_scores, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Puntuación de Silueta Promedio')
plt.title('Análisis de Siluetas')
plt.grid(True)
plt.show()

In [ ]:
# Aplicamos K-Means para clasificar las regiones en grupos
kmeans = KMeans(n_clusters=4, n_init=100, random_state=0).fit(data)

# Asigna un color a cada región en base a los clusters
data['Cluster'] = kmeans.labels_

In [ ]:
# Reducción de dimensionalidad con t-SNE
tsne = TSNE(n_components=2, random_state=0, perplexity=45)
tsne_coordinates = tsne.fit_transform(data)
data['tsne_x'] = tsne_coordinates[:, 0]
data['tsne_y'] = tsne_coordinates[:, 1]

# Generación de colores para los clusters
cluster_colors = plt.cm.tab10.colors
plt.figure(figsize=(12, 8))
unique_clusters = np.unique(kmeans.labels_)
data['provincia_id'] = data['provincia_id']
for cluster in unique_clusters:
    cluster_data = data[data['Cluster'] == cluster]
    plt.scatter(cluster_data['tsne_x'], cluster_data['tsne_y'],
                label=f'Cluster {cluster}', alpha=0.7, s=50)

legend_elements = [Line2D([0], [0], marker='o', color='w', markerfacecolor=f'C{cluster}', markersize=10, label=f'Cluster {cluster}') for cluster in unique_clusters]

# Coloca la leyenda al costado del gráfico
plt.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.05, 1))

plt.title('Clustering con K-Means (' + str(kmeans.n_clusters) +' clústers) y t-SNE ( '+ str(tsne.perplexity) + ' de perplejidad) de Regiones y provincias')
plt.xlabel('t-SNE x')
plt.ylabel('t-SNE y')
plt.show()